In [1]:
#-*-coding:utf-8 -*-
#所有需要的库文件添加该处
import pandas as pd
from six import StringIO
import matplotlib.pyplot as plt
import  numpy as np
import os
import sys
from datetime import datetime,timedelta
from pyecharts import Line,Kline,Bar,Overlap,Grid
from jqdatasdk import *
import requests
import tushare as ts
from dateutil.parser import parse
#import pandas_profiling
pd.set_option('precision',3)


文件所在目录：index_dir = "C:\\quanttime\\data\\index\\"

In [2]:
#所用文件目录如下：
stock_basic_info_dir = "C:\\quanttime\\data\\basic_info\\all_stock_info.csv"
index_basic_info_dir = "C:\\quanttime\\data\\basic_info\\index_all_info_for_analyse.csv"
index_valuation_dir = "C:\\quanttime\\data\\index\\index_valuation\\"
stock_valuation_dir = "C:\\quanttime\\data\\finance\\valuation\\"
index_k_dir = "C:\\quanttime\\data\\index\\jq\\"


In [3]:
#需要分析的指数添加到该list中
analyse_index_list = ["000001.XSHG", "000300.XSHG", "000905.XSHG", "399001.XSHE",\
                      "399005.XSHE", "399006.XSHE", "399016.XSHE","399300.XSHE",\
                      "399975.XSHE","399368.XSHE","399959.XSHE","399967.XSHE","399241.XSHE",\
                      "000015.XSHG","000016.XSHG","000813.XSHG","000814.XSHG","000815.XSHG",\
                     "000005.XSHG","000006.XSHG","000007.XSHG","000015.XSHG","000816.XSHG",\
                     "000948.XSHG","000043.XSHG"]

In [4]:
#加载index 基本信息
df_index_info = pd.read_csv(index_basic_info_dir,index_col=["code"],encoding="gbk")

对指数的估值进行分析

In [5]:

#结果按照如下column排列：
#code，名称，Pb，5%分位，PB_min，PB_max，PB_中值，PB_方差，Pe，5%分位，PE_min，PE_max，PE_均值，PE_方差
general_result = []
for index in analyse_index_list:
    index_valuation = index_valuation_dir + index + '.csv'
    #df_index_valuation = pd.read_csv(index_valuation,index_col=['date'])
    #df_index_valuation = df_index_valuation[~df_index_valuation.reset_index().duplicated().values]
    df_index_valuation = pd.read_csv(index_valuation)
    df_index_valuation = df_index_valuation.drop_duplicates("date")
    df_index_valuation = df_index_valuation.set_index("date")
    
    name = df_index_info.loc[index,['display_name']].display_name
    pb = df_index_valuation.loc[df_index_valuation.index[len(df_index_valuation.index)-1],['PB']].PB
    pb_quantile_5 = df_index_valuation.PB.quantile(0.05)
    pb_min = '%r(%r)'%(df_index_valuation.PB.min(),df_index_valuation.PB.idxmin())
    pb_max = '%r(%r)'%(df_index_valuation.PB.max(),df_index_valuation.PB.idxmax())
    pb_median = df_index_valuation.PB.median()
    pb_std = df_index_valuation.PB.std()
    pe = df_index_valuation.loc[df_index_valuation.index[len(df_index_valuation.index)-1],['PE']].PE
    pe_quantile_5 = df_index_valuation.PE.quantile(0.05)
    pe_min = '%r(%r)'%(df_index_valuation.PE.min(),df_index_valuation.PE.idxmin())
    pe_max = '%r(%r)'%(df_index_valuation.PE.max(),df_index_valuation.PE.idxmax())
    pe_median = df_index_valuation.PE.median()
    pe_std = df_index_valuation.PE.std()
    general_result.append([name,pb,pb_quantile_5,pb_min,pb_max,pb_median,pb_std,pe,pe_quantile_5,pe_min,pe_max,pe_median,pe_std])
df_columns=[u'名称', "PB", u'5%分位', u'PB最小', u'PB最大', u'PB中位数', u'PB标准差', \
                     "PE", u'5%分位', u'PE最小', u'PE最大', u'PE中位数', u'PE标准差']
pe_df = pd.DataFrame(data=general_result,index=analyse_index_list,columns=df_columns)
pe_df.index.name = "code"
pe_df
    
    
    
    

,名称,PB,5%分位,PB最小,PB最大,PB中位数,PB标准差,PE,5%分位,PE最小,PE最大,PE中位数,PE标准差
code,,,,,,,,,,,,,
000001.XSHG,上证指数,1.38,1.310,1.21('2014/5/7'),7.14('2007/10/15'),1.89,1.094,13.39,10.430,9.04('2014/4/28'),72.54('2007/10/15'),17.380,11.400
000300.XSHG,沪深300,1.39,1.290,1.17('2014-05-07'),7.46('2007-10-16'),1.76,1.206,12.25,9.320,8.18('2014-05-19'),65.63('2007-10-31'),14.085,10.242
000905.XSHG,中证500,1.81,1.806,1.44('2008-11-04'),5.89('2015-06-15'),2.77,0.902,26.52,21.406,17.54('2019-01-03'),103.75('2008-01-15'),38.100,19.343
399001.XSHE,深证成指,2.37,1.860,1.67('2005-11-15'),8.82('2007-10-16'),2.80,1.252,23.85,12.486,10.61('2005-07-11'),63.44('2007-10-16'),22.000,11.033
399005.XSHE,中小板指,3.04,2.990,2.39('2008-11-04'),8.17('2007-10-10'),4.15,1.162,24.09,23.072,19.44('2008-10-27'),80.41('2008-01-15'),34.580,11.801
399006.XSHE,创业板指,4.10,3.060,2.51('2012-12-03'),15.01('2015-06-03'),4.71,1.920,49.56,32.782,27.99('2012-12-03'),139.96('2015-06-03'),54.630,19.402
399016.XSHE,深证创新指数,3.15,2.780,2.57('2019-01-03'),4.79('2017-11-13'),4.09,0.574,26.95,19.645,18.33('2019-01-03'),40.29('2017-11-13'),32.875,6.185
399300.XSHE,沪深300,1.39,1.290,1.17('2014-05-07'),7.46('2007-10-16'),1.77,1.206,12.25,9.320,8.18('2014-05-19'),65.63('2007-10-31'),14.110,10.241
399975.XSHE,中证全指证券公司指数(四级行业),1.72,1.340,1.1('2018/10/18'),6.11('2015/6/8'),2.41,0.980,118.20,15.555,12.4('2015/9/15'),145.18('2019-04-04'),38.155,18.759


当日轮动及海龟策略结果

In [6]:
all_trade_day = pd.read_csv("C:\\quanttime\\data\\basic_info\\all_trade_day.csv",index_col=["trade_date"])
result = []
for index1 in df_index_info.index:
    row_record = []
    index_file = index_k_dir + str(index1) + ".csv"
    index_name = df_index_info.loc[index1,["display_name"]].display_name
    #print(index_name)
    row_record.append(index_name)
    try:
        index_k_data = pd.read_csv(index_file,index_col=["date"])
        current_date = index_k_data.index[len(index_k_data.index) - 1]
        row_record.append(current_date)
        #1、与前20天的情况比较，当前持有则低于前20日收盘价则卖出，当前空仓则高于前20日收盘价则买入
        back_trace = 20
        if all_trade_day.index.tolist().index(current_date) - back_trace >= 0:
            pre_date = all_trade_day.index[all_trade_day.index.tolist().index(current_date) - back_trace]
            pre_value = index_k_data.loc[pre_date,['close']].close
            current = index_k_data.loc[current_date,['close']].close
            if float(pre_value) > float(current):
                #compare = u'当前close低于20日前close'
                compare = 'sell'
                row_record.append(compare)
                #print("%r-%r 当前价格低于20日前"%(index1,index_name))
            else:
                #compare = u'当前close高于（或等于）20日前close'
                compare = 'buy'
                row_record.append(compare)
                #print("%r-%r 当前价格高于20日前"%(index1,index_name))
        #2、前20日收盘价，前10日收盘价情况比较，当前持有则低于前10日收盘价则卖出，当前空仓则高于前20日收盘价则买入
            back_trace = 10
            pre_date = all_trade_day.index[all_trade_day.index.tolist().index(current_date) - back_trace]
            if float(current) >= float(pre_value):
                #compare = u'当前close高于等于20日前close'
                compare = 'buy'
                row_record.append(compare)
            elif float(current) < float(index_k_data.loc[pre_date,['close']].close) :
                #compare = u'当前close低于10日前close'
                compare = 'sell'
                row_record.append(compare)
            else:
                #compare = u'当前close介于20日前close与10日前close之间'
                compare = 'hold'
                row_record.append(compare)
        #3、买入与前76日收盘价比较，卖出与33日收盘价比较，即当前close高于前76日close则买入或继续持有，当前close低于前33日close则卖出
        back_trace = 76
        if all_trade_day.index.tolist().index(current_date) - back_trace >= 0:
            pre_date = all_trade_day.index[all_trade_day.index.tolist().index(current_date) - back_trace]
            pre_date_33 = all_trade_day.index[all_trade_day.index.tolist().index(current_date) - 33]
            pre_value = index_k_data.loc[pre_date,['close']].close
            current = index_k_data.loc[current_date,['close']].close
            #print(pre_value)
            #print(current)
            if float(current) >= float(pre_value):
                #compare = u'当前close高于等于76日前close'
                compare = 'buy'
                row_record.append(compare)
            elif float(current) < float(index_k_data.loc[pre_date_33,['close']].close) :
                #compare = u'当前close低于33日前close'
                compare = 'sell'
                row_record.append(compare)
            else:
                #compare = u'当前close介于76日前close与33日前close之间'
                compare = 'hold'
                row_record.append(compare)
    except:
        print("%r process error"%index1)
        continue
    #print(row_record)
    result.append(row_record)
df_columns = ['index_name','date','20buy VS 20sell',u'20buy VS 10sell',u'76buy vs 33sell']
df = pd.DataFrame(data=result,index=df_index_info.index,columns=df_columns)
    
    

'000001.XSHG' process error
'000002.XSHG' process error
'000003.XSHG' process error
'000004.XSHG' process error
'000005.XSHG' process error
'000006.XSHG' process error
'000007.XSHG' process error
'000008.XSHG' process error
'000009.XSHG' process error
'000010.XSHG' process error
'000011.XSHG' process error
'000012.XSHG' process error
'000013.XSHG' process error
'000015.XSHG' process error
'000016.XSHG' process error
'000017.XSHG' process error
'000018.XSHG' process error
'000019.XSHG' process error
'000020.XSHG' process error
'000021.XSHG' process error
'000022.XSHG' process error
'000025.XSHG' process error
'000026.XSHG' process error
'000027.XSHG' process error
'000028.XSHG' process error
'000029.XSHG' process error
'000030.XSHG' process error
'000031.XSHG' process error
'000032.XSHG' process error
'000033.XSHG' process error
'000034.XSHG' process error
'000035.XSHG' process error
'000036.XSHG' process error
'000037.XSHG' process error
'000038.XSHG' process error
'000039.XSHG' proces

'399001.XSHE' process error
'399002.XSHE' process error
'399003.XSHE' process error
'399004.XSHE' process error
'399005.XSHE' process error
'399006.XSHE' process error
'399007.XSHE' process error
'399008.XSHE' process error
'399009.XSHE' process error
'399010.XSHE' process error
'399011.XSHE' process error
'399012.XSHE' process error
'399013.XSHE' process error
'399015.XSHE' process error
'399016.XSHE' process error
'399017.XSHE' process error
'399018.XSHE' process error
'399050.XSHE' process error
'399100.XSHE' process error
'399101.XSHE' process error
'399102.XSHE' process error
'399103.XSHE' process error
'399106.XSHE' process error
'399107.XSHE' process error
'399108.XSHE' process error
'399231.XSHE' process error
'399232.XSHE' process error
'399233.XSHE' process error
'399234.XSHE' process error
'399235.XSHE' process error
'399236.XSHE' process error
'399237.XSHE' process error
'399238.XSHE' process error
'399239.XSHE' process error
'399240.XSHE' process error
'399241.XSHE' proces

'399910.XSHE' process error
'399911.XSHE' process error
'399912.XSHE' process error
'399913.XSHE' process error
'399914.XSHE' process error
'399915.XSHE' process error
'399916.XSHE' process error
'399917.XSHE' process error
'399918.XSHE' process error
'399919.XSHE' process error
'399920.XSHE' process error
'399922.XSHE' process error
'399923.XSHE' process error
'399925.XSHE' process error
'399926.XSHE' process error
'399927.XSHE' process error
'399928.XSHE' process error
'399929.XSHE' process error
'399930.XSHE' process error
'399931.XSHE' process error
'399932.XSHE' process error
'399933.XSHE' process error
'399934.XSHE' process error
'399935.XSHE' process error
'399936.XSHE' process error
'399937.XSHE' process error
'399938.XSHE' process error
'399939.XSHE' process error
'399940.XSHE' process error
'399941.XSHE' process error
'399942.XSHE' process error
'399943.XSHE' process error
'399944.XSHE' process error
'399945.XSHE' process error
'399946.XSHE' process error
'399947.XSHE' proces

## 申万指数分析

In [10]:
#申万指数每日估值的目录
sw_dir = "C:\\quanttime\\data\\index\\sw\\valuation\\"
csv_files = os.listdir(sw_dir)
csv_files.remove('bak')
# 分析的起始时间与结束时间
anlyse_date_start = parse("2018-01-01")
anlyse_date_end = parse("20181231")
list_result = []
for csv_file in csv_files:
    file = sw_dir + csv_file
    fetch_columns = ["date","pb","pe","close","index_code","index_name"]
    df_valuation = pd.read_csv(file, encoding='gbk', index_col=["date"], parse_dates=True, usecols=fetch_columns)
    df_valuation = df_valuation[~df_valuation.reset_index().duplicated().values]
    if df_valuation.empty:
        continue
    analyse_date_range = [x for x in df_valuation.index if anlyse_date_start<=x<=anlyse_date_end]
    # 时间段小于一年不分析
    if len(analyse_date_range) <= 220:
        continue
    start = analyse_date_range[0]
    end = analyse_date_range[-1]
    df_analyse = df_valuation.loc[start:end,fetch_columns]
    increase = (df_analyse.loc[end,["close"]]["close"] - df_analyse.loc[start,["close"]]["close"])/ df_analyse.loc[start,["close"]]["close"]
    index_code = df_analyse.loc[end,["index_code"]]["index_code"]
    index_name = df_analyse.loc[end,["index_name"]]["index_name"]
    pb_mean = df_analyse["pb"].median()
    pb_max = df_analyse["pb"].max()
    pb_min = df_analyse["pb"].min()
    list_result.append([index_code, index_name, increase, pb_mean, pb_max, pb_min, str(start.date()), str(end.date())])
df_result = pd.DataFrame(data=list_result, columns=["index_code", "index_name", "increase", "pb_mean", "pb_max", "pb_min","start", "end"])

    

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [12]:
pd.set_option("display.max_rows", None)
df_result

,index_code,index_name,increase,pb_mean,pb_max,pb_min,start,end
0,801001,申万50,-0.255,1.690,2.15,1.35,2018-01-02,2018-12-28
1,801003,申万Ａ指,-0.308,1.890,2.40,1.49,2018-01-02,2018-12-28
2,801005,申万创业板,-0.315,3.340,3.96,2.46,2018-01-02,2018-12-28
3,801010,农林牧渔,-0.231,2.700,3.45,2.33,2018-01-02,2018-12-28
4,801011,林业,-0.361,2.180,3.16,1.60,2018-01-02,2018-12-28
5,801012,农产品加工,-0.358,2.740,3.20,1.94,2018-01-02,2018-12-28
6,801013,农业综合,-0.348,2.660,3.25,1.92,2018-01-02,2018-12-28
7,801014,饲料,-0.209,2.160,2.90,1.73,2018-01-02,2018-12-28
8,801015,渔业,-0.429,1.940,2.80,1.39,2018-01-02,2018-12-28
9,801016,种植业,-0.353,2.640,3.38,1.98,2018-01-02,2018-12-28


In [16]:
sw_dir = "C:\\quanttime\\data\\index\\sw\\valuation\\"
tmp_file = sw_dir + "851931.csv"
tmp_columns = ["date","pb","pe","close","index_code","index_name"]
df_security = pd.read_csv(tmp_file,encoding='gbk', index_col=["date"], parse_dates=True, usecols=tmp_columns)



In [18]:
df_2010_2015 = df_security.loc["2010-01-06":"2015-11-27",["pb"]]
compare = df_2010_2015["pb"].quantile(0.98)
df_2010_2015[df_2010_2015["pb"]>=compare]

,pb
date,
2014-12-17,4.45
2015-03-23,4.44
2015-04-01,4.42
2015-04-03,4.56
2015-04-07,4.63
2015-04-08,4.85
2015-04-09,4.84
2015-04-10,4.87
2015-04-13,4.82


In [9]:
df_security["pb"].quantile(0.90)

5.456000000000004

In [6]:
tmp={}
if tmp:
    print("not empty")